In [14]:
# 確認各位chrom版本 chrome://settings/help
# 依照自己的版本去下載driver https://chromedriver.chromium.org/downloads
# 安裝selenium 工具包 (pip install selenium), 模擬自動化爬蟲流程
# 如果要對driver 有一些視窗上的操作, 參考 https://www.itread01.com/content/1544483715.html
!pip install selenium

In [15]:
from selenium import webdriver #啟動控制我們的driver 
# from selenium.webdriver.chrome.service import Service
# import sys

# s = Service("chromedriver.exe")
# driver = webdriver.Chrome(service=s)

url = 'https://shopee.tw/'
driver = webdriver.Chrome('chromedriver.exe') #開啟chrome
driver.get(url)

C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':


### 使用seleinum 爬取蝦皮, (主要是因為有些盒子要用滑鼠滾輪滾動,小姐姐才會再給盒子資訊)
### 重點學習如何操作driver 控制網頁取得我們要的盒子
###### 操作思路: 打開網頁 -> 全螢幕 -> 找搜尋欄位 -> 下關鍵字搜尋 ->滾動滑鼠滾輪瀏覽畫面 -> 得到新的盒子資訊....

In [13]:
from selenium import webdriver #啟動控制我們的driver 
from selenium.webdriver.common.keys import Keys #模擬使用鍵盤
from selenium.webdriver.chrome.options import Options # 可以對網頁開啟時,做一些篩選,例如關閉開啟網頁的自動翻譯/詢問
import time
#==================Explicit Waits=====(等待driver運作,直到某個特徵出現為止)========================
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 10):
    global imgs
    #偷偷寫一下,輸入(添加名稱的籃子, driver , 每次向下滑多少像素, 總共滾幾次)
    for each_move in range(run_time):
        js = f'window.scrollBy(0,{pix})'
        driver.execute_script(js)
        print('滑鼠滾動一次~')
        time.sleep(1)
        prods = driver.find_elements_by_class_name('_10Wbs-')
        imgs  = driver.find_elements_by_class_name('_3-N5L6')
#         print(imgs)
        #prods 就是當下畫面有的盒子資訊
        for p in prods:
            t = p.text
            if t not in title_list: 
                #每次滾動 會有重複的內容, 做一個判斷,如果有重複的文字在已經append的籃子, 那就不要走這個判斷式
                title_list.append(t)
        for img in imgs:
            link = img.get_attribute('src') #類似bs4 裡面的get()
#             print(link)
        time.sleep(1)
        print('每次滾完滑鼠,看一下籃子裡面的數量:',len(title_list))
    print('已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點')
#     js = f'window.scrollBy(0,-300)'
#     driver.execute_script(js)    
        



chrom_driver_path = 'chromedriver.exe'
url = 'https://shopee.tw/'
item = '包包'
opt = Options()
opt.add_argument('--disable-notifications') #禁止彈出詢問視窗

driver = webdriver.Chrome(chrom_driver_path, chrome_options = opt)
driver.get(url) # 得到url資訊

#放大全螢幕
driver.maximize_window()
# 做一個彈性的等待(因為driver在處理時要時間,不想用sleep去固定時間等待)
# 最多等10秒, 直到畫面有'stardust-popover__target' 出現後才會繼續走程式碼
WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'stardust-popover__target') ) )

# 找搜尋欄位 並且輸入我們想要的搜尋物件
# search = driver.find_element_by_class_name('shopee-searchbar-input__input') #類似bs4 find
search = driver.find_elements_by_class_name('shopee-searchbar-input__input') #類似bs4 find_all
print('!!!!',search)
keyin_bar = search[0]
keyin_bar.send_keys(item)
keyin_bar.send_keys(Keys.ENTER)

#等待盒子的class出現,最多等10秒
WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'col-xs-2-4') ) )
print('已經搜尋完畢..')
# 開始尋找商品文字盒子(_10Wbs-)
# prods = driver.find_elements_by_class_name('_10Wbs-')
# print(prods)
title_list = []
mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 8)
time.sleep(1)
for page in range(2,4):
    buttom_xpath = f'/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[{page+1}]'
    driver.find_element_by_xpath(buttom_xpath).click() #用畫面的絕對位置,去找按鈕,並且點擊
    #進入下一頁
    WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'_10Wbs-') ) )
    #成功跳轉到下一頁
    mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 8)
    print(f'==========第{page}頁完成爬取===========')
driver.quit()

C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: use options instead of chrome_options
C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:56: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


!!!! [<selenium.webdriver.remote.webelement.WebElement (session="79a0396971aa1ed5df6e59a54ae6e193", element="78564548-4f31-49e0-b256-738dc4ff3ecf")>]
已經搜尋完畢..
滑鼠滾動一次~


C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 0
已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點


C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[3]"}
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A8DF13+2219795]
	Ordinal0 [0x00A22841+1779777]
	Ordinal0 [0x0093423D+803389]
	Ordinal0 [0x00963025+995365]
	Ordinal0 [0x009631EB+995819]
	Ordinal0 [0x00990F52+1183570]
	Ordinal0 [0x0097E844+1108036]
	Ordinal0 [0x0098F192+1175954]
	Ordinal0 [0x0097E616+1107478]
	Ordinal0 [0x00957F89+950153]
	Ordinal0 [0x00958F56+954198]
	GetHandleVerifier [0x00D82CB2+3040210]
	GetHandleVerifier [0x00D72BB4+2974420]
	GetHandleVerifier [0x00B26A0A+565546]
	GetHandleVerifier [0x00B25680+560544]
	Ordinal0 [0x00A29A5C+1808988]
	Ordinal0 [0x00A2E3A8+1827752]
	Ordinal0 [0x00A2E495+1827989]
	Ordinal0 [0x00A380A4+1867940]
	BaseThreadInitThunk [0x7712FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77E47A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77E47A6E+238

# 如何調用seleinum 跟bs4做結合使用(範例介紹)

In [ ]:
from bs4 import BeautifulSoup
chrom_driver_path = './chromedriver.exe'
url = 'https://shopee.tw/'
item = '包包'
opt = Options()
opt.add_argument('--disable-notifications') #禁止彈出詢問視窗

driver = webdriver.Chrome(chrom_driver_path, chrome_options = opt)
driver.get(url) # 得到url資訊
WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'stardust-popover__target') ) )

# 找搜尋欄位 並且輸入我們想要的搜尋物件
# search = driver.find_element_by_class_name('shopee-searchbar-input__input') #類似bs4 find
search = driver.find_elements_by_class_name('shopee-searchbar-input__input') #類似bs4 find_all
keyin_bar = search[0]
keyin_bar.send_keys(item)
keyin_bar.send_keys(Keys.ENTER)
time.sleep(5)
js = 'document.documentElement.scrollTop=10000'
driver.execute_script(js)
#以BeautifulSoup偵測selenium的driver 對當下瀏覽去去找當下的盒子
soup = BeautifulSoup(driver.page_source,'html.parser')
bs = soup.find_all('button',class_='shopee-button-no-outline')
bs

# 把上述selenium包成一個class funs

In [3]:

from selenium import webdriver #啟動控制我們的driver 
from selenium.webdriver.common.keys import Keys #模擬使用鍵盤
from selenium.webdriver.chrome.options import Options # 可以對網頁開啟時,做一些篩選,例如關閉開啟網頁的自動翻譯/詢問
import time
#==================Explicit Waits=====(等待driver運作,直到某個特徵出現為止)========================
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 10):
    global imgs
    #偷偷寫一下,輸入(添加名稱的籃子, driver , 每次向下滑多少像素, 總共滾幾次)
    for each_move in range(run_time):
        js = f'window.scrollBy(0,{pix})'
        driver.execute_script(js)
        print('滑鼠滾動一次~')
        time.sleep(1)
        prods = driver.find_elements_by_class_name('_10Wbs-')
        imgs  = driver.find_elements_by_class_name('_3-N5L6')
#         print(imgs)
        #prods 就是當下畫面有的盒子資訊
        for p in prods:
            t = p.text
            if t not in title_list: 
                #每次滾動 會有重複的內容, 做一個判斷,如果有重複的文字在已經append的籃子, 那就不要走這個判斷式
                title_list.append(t)
        for img in imgs:
            link = img.get_attribute('src') #類似bs4 裡面的get()
#             print(link)
        time.sleep(1)
        print('每次滾完滑鼠,看一下籃子裡面的數量:',len(title_list))
    print('已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點')
#     js = f'window.scrollBy(0,-300)'
#     driver.execute_script(js)    
        

class shopee:
    def __init__(self,):
        self.chrom_driver_path = './chromedriver.exe'
        self.opt = Options()
        self.opt.add_argument('--disable-notifications') #禁止彈出詢問視窗
        self.driver = webdriver.Chrome(self.chrom_driver_path, chrome_options = self.opt)
        
        self.pix = 500
        self.run_time = 8
        self.title_list = []
    
    def page_crawl(self , item = '包包'):
        url = 'https://shopee.tw/'
        self.driver.get(url) # 得到url資訊
        #放大全螢幕
        self.driver.maximize_window()

        # 做一個彈性的等待(因為driver在處理時要時間,不想用sleep去固定時間等待)
        # 最多等10秒, 直到畫面有'stardust-popover__target' 出現後才會繼續走程式碼
        WebDriverWait(self.driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'stardust-popover__target') ) )

        # 找搜尋欄位 並且輸入我們想要的搜尋物件
        # search = driver.find_element_by_class_name('shopee-searchbar-input__input') #類似bs4 find
        search = self.driver.find_elements_by_class_name('shopee-searchbar-input__input') #類似bs4 find_all
        keyin_bar = search[0]
        keyin_bar.send_keys(item)
        keyin_bar.send_keys(Keys.ENTER)

        #等待盒子的class出現,最多等10秒
        WebDriverWait(self.driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'col-xs-2-4') ) )
        print('已經搜尋完畢..')

        mouse_roll_and_crawl( self.title_list , self.driver ,pix =self.pix , run_time = self.run_time)
        time.sleep(1)
        for page in range(2,3):
            buttom_xpath = f'/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[{page+1}]'
            self.driver.find_element_by_xpath(buttom_xpath).click() #用畫面的絕對位置,去找按鈕,並且點擊
            #進入下一頁
            WebDriverWait(self.driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'_10Wbs-') ) )
            #成功跳轉到下一頁
            mouse_roll_and_crawl( self.title_list , self.driver ,pix =self.pix , run_time = self.run_time)
            print(f'==========第{page}頁完成爬取===========')
        self.driver.quit()
        return self.title_list

In [4]:
from shopee_funs import mouse_roll_and_crawl , shopee
my_item   = str(input('請輸入要爬取的物件(ex:包包):'))
save_path = str(input('請輸入要存取的路徑以及檔名(ex:./bag.txt):'))
#調用上述的class
S = shopee() #此調用動作已經包含了 init 裡面開啟chrome的功能
crawl_list = S.page_crawl(item=my_item)
crawl_list

output_file = open(save_path,'w',encoding = 'utf8')
for each_title in crawl_list:
    output_file.write(str(each_title)+'\n')
output_file.close()

C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.chrom_driver_path, chrome_options = self.opt)
C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:42: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(self.chrom_driver_path, chrome_options = self.opt)
C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:60: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  search = self.driver.find_elements_by_class_name('shopee-searchbar-input__input') #類似bs4 find_all


已經搜尋完畢..
滑鼠滾動一次~


C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  prods = driver.find_elements_by_class_name('_10Wbs-')
C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  imgs  = driver.find_elements_by_class_name('_3-N5L6')


每次滾完滑鼠,看一下籃子裡面的數量: 11
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 21
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 26
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 36
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 41
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 51
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 56
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 61
已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點


C:\Users\10903059\AppData\Local\Temp/ipykernel_26056/2665925799.py:73: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  self.driver.find_element_by_xpath(buttom_xpath).click() #用畫面的絕對位置,去找按鈕,並且點擊


滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 71
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 81
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 86
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 96
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 101
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 110
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 113
滑鼠滾動一次~
每次滾完滑鼠,看一下籃子裡面的數量: 117
已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點
==========第2頁完成爬取===========


['[台灣現貨]【簡約學院風】🔥尼龍後背包 後背包 韓系 日系 雙肩包 背包 旅行包 電腦包 多功能包 休閒包 書包 後背包',
 '[台灣現貨]【純色14吋尼龍筆電後背包】🔥韓系 日系 雙肩包 背包 旅行包 電腦包 多功能包 休閒包 書包',
 '🔥台灣現貨秒發🔥 ins 潮流工裝後背包 日系 黑色 原宿 雙肩 學生 背包 百搭 書包 後背包 復古風',
 '【台灣現貨☆免運☆】【背部防盜15.6吋尼龍筆電後背包】韓系日系 雙肩包 背包 電腦包 多功能包 書包 包郵',
 '台灣出貨🔥防水斜背包/側背包 大容量胸包 密碼防盜背包 可外置充電/男士肩包/單肩包 側包/斜包 男 斜挎包/側肩包',
 '【免運/🇹🇼公司貨附發票】耐重防雨多隔層後背包/尼龍防潑水多層次休閒背包/學生書包/14吋筆電背包',
 '台灣快速出貨🚄 日系簡約 可裝15吋筆電 工裝風 大容量 防盜後背包 多隔層防潑水 雙肩包 大學生背包 校園書包',
 '免運🚚【🇹🇼公司貨附發票】運動休閒大容量後背包 耐重多隔層背包 多層次15.6吋筆電背包 商務背包',
 '台灣快速出貨🚄 防水斜背包 側背包 大容量胸包 防盜背包 斜包 充電肩背包 單肩包 男 斜挎包 男女運動側肩包 側包',
 '簡約後背包 日系背包 學院風後背包 ins風大容量高中學生書包女韓版原宿ulzzang卡通雙肩包國中高中背包',
 '',
 '【出貨x免運】ins風2022新款韓版Chill男女背包大學生背包 女包防水 森系百搭防水耐重後背包大容量簡約港風雙肩包',
 '筆電後背包 撞色電腦包 減壓設計 超大空間 USB充電插孔 多層夾層 可適用 13吋 14吋 15.6吋 筆電',
 'Mia💙韓 ins 防水 工業風背包 古著感 後背包 旅行後背 大容量 書包(๑´ㅂ`๑) 雙肩包',
 '【年後出貨/聊聊送零錢包】Who a u 後背包 新款書包 抽繩後背包 零錢包 韓國書包 大容量 who.a.u',
 '潮牌原創情侶休閒雙肩包 純色後背包 肩背包男大容量帆布高中學生書包滑板包男士旅行背包 百搭男包 背包時尚個性男包',
 '【Bfuming】日系簡約 可裝15吋筆電 工裝風 大容量 防盜後背包 多隔層防潑水 雙肩包 大學生背包 校園書包',
 '夜光大折扣新款時尚雙肩包男包背包 旅行休閑電腦包大學中學生書包',
 '【

In [5]:
# 把上述功能打包成exe 
# 相依套件: pip install auto-py-to-exe
# !pip install auto-py-to-exe
# step 1: 將主要文件寫成py檔來執行 ,確定py檔有互相調用
# step 2: 直接去網路下載一個想要的圖片(檔案附檔名是ico)(https://cloudconvert.com/jpg-to-ico)
# step3 : 在cmd 去移動路徑到目前主要py檔的資料夾中, 輸入auto-py-to-exe